# Extracting Text from Images with Python and Pytesseract: Enhanced Tutorial

## Introduction to OCR with Pytesseract

**Purpose:**
Optical Character Recognition (OCR) converts images of text into machine-readable strings. Pytesseract is a Python wrapper for Google’s Tesseract-OCR engine, enabling seamless integration with Python image-processing libraries. This tutorial covers installation, fundamental workflows, advanced preprocessing, configuration, and best practices to maximize OCR accuracy.

**Why OCR Matters:**
- **Digitization**: Transform scanned documents, PDFs, and photographs into searchable text
- **Data Extraction**: Automate data entry from invoices, forms, and reports
- **Accessibility**: Convert visual text to speech or Braille for visually impaired users
- **Analytics**: Enable text analytics on image-based content (e.g., signage, social media images)

**Common OCR Challenges:**
- Poor image quality (low resolution, noise)
- Complex layouts (multiple text blocks, tables)
- Non-uniform lighting and skewed text
- Varied fonts, sizes, and languages

## 1. Installation and Setup

### Tesseract Engine

**Windows:** Download the MSI installer from the UB Mannheim repository and note the install path (e.g., `C:\Program Files\Tesseract-OCR`). Add it to your `PATH` or configure `pytesseract.pytesseract.tesseract_cmd`.

**Linux/macOS:**
```bash
sudo apt install tesseract-ocr      # Debian/Ubuntu
brew install tesseract             # macOS with Homebrew
```

### Python Dependencies

```bash
pip install pytesseract pillow opencv-python matplotlib
```

**Verification:**
```python
import pytesseract
print(pytesseract.get_tesseract_version())
```

## 2. Basic OCR Workflow

**Purpose:**
Quickly extract text from a clean, high-resolution image.


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pytesseract

#download image
!wget "https://muidsi.missouri.edu/wp-content/uploads/2023/11/F_Edu_Best-Masters-Data-Science_top10-US_2023.png" -O sample_document.png
# Load an image from file
image = Image.open('sample_document.png')
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Extract text using pytesseract
text = pytesseract.image_to_string(image)
print(text)

**Notes:**
- Defaults use LSTM engine (`--oem 3`) and automatic page segmentation (`--psm 3`).
- Accuracy heavily depends on image quality and preprocessing.

## 3. Advanced Image Preprocessing

**Purpose:**
Enhance image quality to reduce noise and improve Tesseract’s recognition accuracy.

| Step                | Description                                      | OpenCV Example                                           |
|---------------------|--------------------------------------------------|----------------------------------------------------------|
| Grayscale           | Simplifies image, reduces color noise            | `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`                 |
| Resize              | Upscale small text regions                       | `cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)` |
| Noise Removal       | Smooth out artifacts                              | `cv2.medianBlur(gray, 5)`                               |
| Contrast Enhancement| Improve text-background contrast                  | `clahe = cv2.createCLAHE(clipLimit=2); clahe.apply(gray)`|
| Thresholding        | Binarize image for clear text regions             | `cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)`|
| Adaptive Threshold  | Local threshold for uneven lighting               | `cv2.adaptiveThreshold`                                  |
| Deskew              | Correct tilted text                              | Custom deskew function (below)                          |
| Morphology          | Dilation/erosion to connect/disconnect components| `cv2.dilate` / `cv2.erode`                               |

### Deskewing Function

In [ ]:
import numpy as np
import cv2

def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

### Preprocessing Pipeline Example

In [ ]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt

def preprocess_image(
    image_path,
    grayscale=True,
    resize=True,
    blur=True,
    threshold=False,  # Turn off by default
    adaptive_threshold=False,
    clahe=True,
    resize_fx=2,
    resize_fy=2,
    blur_ksize=(5, 5)
):
    """
    Preprocess the image with optional steps.
    """
    img = cv2.imread(image_path)
    if grayscale:
        # Convert to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if resize:
        # Resize image to enlarge text
        img = cv2.resize(img, None, fx=resize_fx, fy=resize_fy,
                         interpolation=cv2.INTER_CUBIC)
    if blur:
        # Apply Gaussian blur to reduce noise
        img = cv2.GaussianBlur(img, blur_ksize, 0)
    if clahe:
        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe_obj = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img = clahe_obj.apply(img)
    if threshold:
        # Apply global thresholding (Otsu's method)
        _, img = cv2.threshold(
            img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if adaptive_threshold:
        # Apply adaptive thresholding
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, 10)
    return img

# Load an image from file using PIL for display
image = Image.open('sample_course_path.png')
# Preprocess the image using the defined function
preprocessed_img = preprocess_image('sample_course_path.png')

# Display original and preprocessed images side by side
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(image)
ax[0].set_title('Original Image')
ax[0].axis('off')

ax[1].imshow(preprocessed_img, cmap='gray')
ax[1].set_title('Preprocessed Image')
ax[1].axis('off')

plt.tight_layout()
plt.show()

# Extract text from the original image using pytesseract
text = pytesseract.image_to_string(image)
print(text)

# Extract text from the preprocessed image with custom config
text = pytesseract.image_to_string(preprocessed_img)
print(text)


## 4. Custom OCR Configuration

**Purpose:**
Fine-tune Tesseract parameters for your image’s layout and content.


In [ ]:
# Use custom OCR configuration: --oem 3 (LSTM OCR Engine), --psm 6 (Assume a single uniform block of text)
config = '--oem 3 --psm 6'
text = pytesseract.image_to_string(image, config=config)
print(text)

# Note: This configuration works best for images with a single block of text.
# Our sample image may contain complex layouts or graphics, so results may vary.

**Common Flags:**
- `--oem [0–3]`: OCR engine mode (legacy, LSTM, combined)
- `--psm [0–13]`: Page segmentation mode (single line, block, sparse text)
- `-c tessedit_char_whitelist=ABC...`: Restrict character set

## 5. Multilingual OCR

**Purpose:**
Recognize text in multiple languages or scripts.

In [ ]:
# English + French
text = pytesseract.image_to_string(preprocessed_img, lang='eng+fra')
print(text)

## 6. Postprocessing and Error Correction

**Purpose:**
Clean and correct OCR output using heuristics or NLP techniques.

In [ ]:
import re
import pytesseract

def clean_ocr(text):
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Fix common OCR misrecognitions
    corrections = {
        '0': 'O',
        '1': 'I',
        '5': 'S'
    }
    for k, v in corrections.items():
        text = text.replace(k, v)
    
    return text

# Example use — assuming `pre` is an image processed for OCR
raw = pytesseract.image_to_string(preprocessed_img)
print(clean_ocr(raw))

## 7. Alternative OCR Libraries

- **EasyOCR**: Multilingual, supports 80+ languages, handwriting
- **OCRmyPDF**: OCR PDFs and embed text layer
- **Google Cloud Vision**: High accuracy, cloud-based

## 8. Best Practices and Troubleshooting

- **High DPI images** (>300 DPI) for clarity
- **Consistent lighting and contrast** to reduce noise
- **Avoid overly complex layouts** or split into separate images
- **Experiment with preprocessing combinations** (resize + threshold + deskew)
- **Use ROI cropping** to focus OCR on relevant areas
- **Profile performance** on sample images before batch processing

## Conclusion

Optimizing OCR involves a combination of image preprocessing, Tesseract configuration, and postprocessing. This tutorial equips you to build robust OCR pipelines, from basic extraction to advanced, domain-specific solutions. Experiment, measure accuracy, and iterate to achieve the best results for your use case.